In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.chdir('/kaggle/input/bike-sharing-dataset')

In [ ]:
df_original = pd.read_csv('day.csv')

In [ ]:
df = df_original.copy()
df.head(2)

In [ ]:
print('Number of Variables: '+str(len(df.columns)))
print('Variables:'+str(list(df.columns)))
print('Shape of dataframe '+ str(df.shape))

In [ ]:
#Hypothesis, sum of casual and registered should be equal to cnt
#Verify our hypothesis
j=0;
for i in range(len(df)):
    if df['cnt'].loc[i] == df['casual'].loc[i] + df['registered'].loc[i]:
        pass;
    else:
        print('not equal')

#Since it did not print "not equal", our hypothesis is correct

Model Type : Regression
Objective:

Predict 'casual' and 'registered'
Add both the predicted value to get 'cnt'
Display value of 'cnt' to user

In [ ]:
df.info()

In [ ]:
# Missing Value Analysis
df.isna().sum()

In [ ]:

#Categorical Variable
print(df['season'].value_counts())
print(df['yr'].value_counts())
print(df['mnth'].value_counts())
print(df['holiday'].value_counts())
print(df['weekday'].value_counts())
print(df['workingday'].value_counts())
print(df['weathersit'].value_counts())

#Continuous numberical variable
#print(df['temp'].value_counts())
#print(df['atemp'].value_counts())
#print(df['hum'].value_counts())
#print(df['windspeed'].value_counts())

In [ ]:
#Convert Columns into their respective types
df['dteday'] = pd.to_datetime(df['dteday'])

df['season'] = df['season'].astype('category')
df['yr'] = df['yr'].astype('category')
df['mnth'] = df['mnth'].astype('category')
df['holiday'] =df['holiday'].astype('category')
df['weekday'] = df['weekday'].astype('category')
df['workingday'] = df['workingday'].astype('category')
df['weathersit'] = df['weathersit'].astype('category')

In [ ]:
df.describe() 

In [ ]:
df = df.rename(columns={'instant':'record_index','hum':'humidity','cnt':'total_count','weathersit':'weather_category','mnth':'month','dteday':'date'})

In [ ]:
fig, axs  = plt.subplots(2, 5, figsize=(20, 10))

axs[0,0].boxplot(df['yr'])
axs[0,0].set_title('Year', size=20)
axs[0,1].boxplot(df['month'])
axs[0,1].set_title('Month', size=20)
axs[0,2].boxplot(df['weekday'])
axs[0,2].set_title('Weekday', size=20)
axs[0,3].boxplot(df['workingday'])
axs[0,3].set_title('Working Day', size=20)
axs[0,4].boxplot(df['weather_category'])
axs[0,4].set_title('Weather Category', size=20)
axs[1,0].boxplot(df['temp'])
axs[1,0].set_title('Temp', size=20)
axs[1,1].boxplot(df['atemp'])
axs[1,1].set_title('atemp', size=20)
axs[1,2].boxplot(df['humidity'])
axs[1,2].set_title('Humidity', size=20)
axs[1,3].boxplot(df['windspeed'])
axs[1,3].set_title('Wind Speed', size=20)
axs[1,4].boxplot(df['season'])
axs[1,4].set_title('Season', size=20)
plt.show()

In [ ]:
c_category = ['season','yr','month','holiday','weekday','workingday','weather_category']

In [ ]:
c_continuous = ['temp','atemp','hum','windspeed']

In [ ]:
cname = [x for x in df.columns if x not in ['date','registered','casual','cnt','instant','season','yr','month','holiday','weekday','workingday','weather_category']]
cname

In [ ]:
#Outlier Analysis, box plot ----------> Remove outlier
for col in cname:
    q75,q25 = np.percentile(df.loc[:,col],[75,25]);
    iqr = q75-q25;
    lower_fence = q25 - (1.5*iqr)
    upper_fence = q75 + (1.5*iqr)

    df_outlier_del = df.drop(df[df.loc[:,col] < lower_fence].index)
    df_outlier_del = df.drop(df[df.loc[:,col] > upper_fence].index)

In [ ]:
#Outlier Analysis by considering outliers as NA and imputing values
df_outlier_impute = df.copy()
for col in cname:
    q75,q25 = np.percentile(df.loc[:,col],[75,25]);
    iqr = q75-q25;
    lower_fence = q25 - (1.5*iqr)
    upper_fence = q75 + (1.5*iqr)

    row = df_outlier_impute[df.loc[:,col] < lower_fence].index
    df_outlier_impute.loc[row,col] = np.nan

    row = df_outlier_impute[df.loc[:,col] > upper_fence].index
    df_outlier_impute.loc[row,col] = np.nan

In [ ]:
df_outlier_impute.isna().sum()

In [ ]:
print('Percentage of Missing value:',(15/731)*100)

In [ ]:
#To check the similarities, lets replace df['weekday][0] = 6
df_outlier_impute.loc[0,'temp'] = np.nan
df_outlier_impute.loc[0,'temp']

In [ ]:
#Original = 0.344
#Mean = 0.49
#Median = 0.49
#KNN = 0.37

#df_outlier_impute['temp'] = df_outlier_impute['temp'].fillna(df_outlier_impute['temp'].mean())
#print('Mean: ',df_outlier_impute.loc[0,'temp'])


#df_outlier_impute['temp'] = df_outlier_impute['temp'].fillna(df_outlier_impute['temp'].median())
#print('Median: ',df_outlier_impute.loc[0,'temp'])

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_fit = imputer.fit_transform(df_outlier_impute[['temp', 'atemp', 'humidity', 'windspeed']])
df_fit = pd.DataFrame(df_fit, columns = ['temp', 'atemp', 'humidity', 'windspeed'])
print('KNN: ',df_fit.loc[0,'temp'])

df_outlier_impute[['temp', 'atemp', 'humidity', 'windspeed']]=df_fit[['temp', 'atemp', 'humidity', 'windspeed']]

a) Imputed Values

In [ ]:
#Correlation plot(only between continuous numerical variable)
fx, ax = plt.subplots(figsize = (10,7))
num_var = df_outlier_impute[['temp','atemp','humidity','windspeed','casual','registered','total_count']]
num_var.corr()
sns.heatmap(num_var.corr(),annot=True,cmap='Blues', linewidth=2)

Observation:

all variable except "humidity" got correlation more than 0.1 0
temp and atemp got high correlation of 0.99 i.e. we can discard one of the variable
registereed and cnt got high correlation

In [ ]:
from scipy.stats import chi2_contingency
#Catigorical variable
c_category = ['season','yr','month','holiday','weekday','workingday','weather_category']

print('------------FOR CASUAL----------------------')
for col_name in c_category:
    print(col_name)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df_outlier_impute['casual'],df_outlier_impute[col_name])) 
    print('p :',p)
print('-------------------FOR REGISTERED----------------------')
for col_name in c_category:
    print(col_name)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df_outlier_impute['registered'],df_outlier_impute[col_name]))
    print('p :',p)

print('------------------FOR Total Count----------------------')
for col_name in c_category:
    print(col_name)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df_outlier_impute['total_count'],df_outlier_impute[col_name]))
    print('p :',p)


if p < 0.05 then there is high correlation else it's independent

1. Holiday got no affect on both casual or registered
2. Other than 'season' and 'month' for predicting 'casual', every variable got p value greater than 0.05 i.e they are independent of the target variable


**Observation:**
1. Users which are registred are not affected by any weather, month, year, weekday, working/weekday.
2. Casual users count are affected by Season and month the most.


b) Drop Value DataFrame

In [ ]:
fx, ax = plt.subplots(figsize = (10,7))
num_var = df_outlier_del[['temp','atemp','humidity','windspeed','casual','registered','total_count']]
num_var.corr()
sns.heatmap(num_var.corr(),annot=True,cmap='Blues', linewidth=2)

In [ ]:
#Catigorical variable
c_category = ['season','yr','month','holiday','weekday','workingday','weather_category']

print('------------FOR CASUAL----------------------')
for col_name in c_category:
    print(col_name)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df_outlier_del['casual'],df_outlier_del[col_name])) 
    print('p :',p)
print('-------------------FOR REGISTERED----------------------')
for col_name in c_category:
    print(col_name)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df_outlier_del['registered'],df_outlier_del[col_name]))
    print('p :',p)

print('------------------FOR Total Count----------------------')
for col_name in c_category:
    print(col_name)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df_outlier_del['total_count'],df_outlier_del[col_name]))
    print('p :',p)


Observation:
There is not much change in correlation value between dependent and independent values.

Since I want to preserve data, I will go with imputation of data on outliers

------------------------------------------------DATA VISUALIZATION---------------------------------

In [ ]:
# Season

fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(10,5))
sns.barplot(x="season", y="casual", data=df_outlier_impute, ci=None, ax=axs[0,0])
axs[0,0].set_title('Casual')
sns.barplot(x="season", y="registered", data=df_outlier_impute, ci=None,ax=axs[0,1])
axs[0,1].set_title('Registered')
sns.scatterplot(x = "season", y = "casual",data=df_outlier_impute, ax=axs[1,0])
sns.scatterplot(x = "season", y = "registered",data=df_outlier_impute, ax=axs[1,1])
#1: Spring, 2: Summer, 3: fall, 4:winter

In [ ]:
#Year

fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(10,5))
sns.barplot(x="yr", y="casual", data=df_outlier_impute, ci=None,ax=axs[0,0])
axs[0,0].set_title('Casual')
sns.barplot(x="yr", y="registered", data=df_outlier_impute, ci=None,ax=axs[0,1])
axs[0,1].set_title('Registered')
sns.scatterplot(x = "yr", y = "casual",data=df_outlier_impute, ax=axs[1,0])
sns.scatterplot(x = "yr", y = "registered",data=df_outlier_impute, ax=axs[1,1])
# 1: 2012, 0: 2011

In [ ]:
#Month

fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(10,5))
sns.barplot(x="month", y="casual", data=df_outlier_impute, ci=None, hue='yr',ax=axs[0,0])
axs[0,0].set_title('Casual')
sns.barplot(x="month", y="registered", data=df_outlier_impute, ci=None,hue='yr',ax=axs[0,1])
axs[0,1].set_title('Registered')
sns.scatterplot(x = "month", y = "casual",data=df_outlier_impute, ax=axs[1,0])
sns.scatterplot(x = "month", y = "registered",data=df_outlier_impute, ax=axs[1,1])

In [ ]:
#Weekday

fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(10,5))
sns.barplot(x="weekday", y="casual", data=df_outlier_impute, ci=None,ax=axs[0,0])
axs[0,0].set_title('Casual')
sns.barplot(x="weekday", y="registered", data=df_outlier_impute, ci=None,ax=axs[0,1])
axs[0,1].set_title('Registered')
sns.scatterplot(x = "weekday", y = "casual",data=df_outlier_impute, ax=axs[1,0])
sns.scatterplot(x = "weekday", y = "registered",data=df_outlier_impute, ax=axs[1,1])

In [ ]:
#Holiday

fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(10,5))
sns.barplot(x="holiday", y="casual", data=df_outlier_impute, ci=None,ax=axs[0,0])
axs[0,0].set_title('Casual')
sns.barplot(x="holiday", y="registered", data=df_outlier_impute, ci=None,ax=axs[0,1])
axs[0,1].set_title('Registered')
sns.scatterplot(x = "holiday", y = "casual",data=df_outlier_impute, ax=axs[1,0])
sns.scatterplot(x = "holiday", y = "registered",data=df_outlier_impute, ax=axs[1,1])
#1: Holiday
#0: No holiday

In [ ]:
#Working Day

fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(10,5))
sns.barplot(x="workingday", y="casual", data=df_outlier_impute, ci=None,ax=axs[0,0])
axs[0,0].set_title('Casual')
sns.barplot(x="workingday", y="registered", data=df_outlier_impute, ci=None,ax=axs[0,1])
axs[0,1].set_title('Registered')
sns.scatterplot(x = "workingday", y = "casual",data=df_outlier_impute, ax=axs[1,0])
sns.scatterplot(x = "workingday", y = "registered",data=df_outlier_impute, ax=axs[1,1])
#0: weekend or holiday
#1: Working day or weekdays

In [ ]:
#Weather Category

fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(10,5))
sns.barplot(x="weather_category", y="casual", data=df_outlier_impute, ci=None,ax=axs[0,0])
axs[0,0].set_title('Casual')
sns.barplot(x="weather_category", y="registered", data=df_outlier_impute, ci=None,ax=axs[0,1])
axs[0,1].set_title('Registered')
sns.scatterplot(x = "weather_category", y = "casual",data=df_outlier_impute, ax=axs[1,0])
sns.scatterplot(x = "weather_category", y = "registered",data=df_outlier_impute, ax=axs[1,1])

#1:Clear, Few clouds, Partly cloudy, Partly cloudy
#2:Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
#3:Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
#4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog  

Scatter plot between numerical continuous variable and target variable

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,5))
sns.scatterplot(x = "temp", y = "casual",data=df_outlier_impute, ax=axs[0])
sns.scatterplot(x = "temp", y = "registered",data=df_outlier_impute, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,5))
sns.scatterplot(x = "atemp", y = "casual",data=df_outlier_impute, ax=axs[0])
sns.scatterplot(x = "atemp", y = "registered",data=df_outlier_impute, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,5))
sns.scatterplot(x = "humidity", y = "casual",data=df_outlier_impute, ax=axs[0])
sns.scatterplot(x = "humidity", y = "registered",data=df_outlier_impute, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(10,5))
sns.scatterplot(x = "windspeed", y = "casual",data=df_outlier_impute, ax=axs[0])
sns.scatterplot(x = "windspeed", y = "registered",data=df_outlier_impute, ax=axs[1])

In [ ]:
sns.scatterplot(x = "casual", y = "registered",data=df_outlier_impute)

In [ ]:
x_casual = df_outlier_impute[['season', 'atemp', 'humidity', 'windspeed']]
x_registered = df_outlier_impute[['atemp', 'humidity', 'windspeed']]
y_casual = df_outlier_impute['casual']
y_registered = df_outlier_impute['registered']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor  
from sklearn.tree import export_graphviz

In [ ]:
def MAPE(y_true,y_pred):
    mape = np.mean(np.abs((y_true - y_pred)/ y_true))
    return mape

def MAE(y_true,y_pred):
    mae = np.mean(np.abs(y_true - y_pred))
    return mae

In [ ]:
x_train_casual, x_test_casual, y_train_casual, y_test_casual = train_test_split(x_casual, y_casual, test_size = 0.4, random_state = 0)

In [ ]:
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(x_train_casual, y_train_casual)

In [ ]:
regressor.score(x_train_casual,y_train_casual)

In [ ]:
y_predicted_d_casual = regressor.predict(x_test_casual)

In [ ]:
regressor.feature_importances_

In [ ]:
mse = MAE(y_test_casual,y_predicted_d_casual)
mape = MAPE(y_test_casual,y_predicted_d_casual)
print("Mean Squared Error:",mse)
print("Mean Absolute Percentage Error:",mape)
print("Accuracy : ",100-mape)

Model for registered

In [ ]:
x_train_registered, x_test_registered, y_train_registered, y_test_registered = train_test_split(x_registered, y_registered, test_size = 0.4, random_state = 0)

In [ ]:
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(x_train_registered, y_train_registered)

In [ ]:
y_predicted_d_registered = regressor.predict(x_test_registered)

In [ ]:
mse = MAE(y_test_registered,y_predicted_d_casual)
mape = MAPE(y_test_registered,y_predicted_d_casual)
print("Mean Squared Error:",mse)
print("Mean Absolute Percentage Error:",mape)

In [ ]:
x = df_outlier_impute.drop(columns=['total_count'], axis=1)
y = df_outlier_impute['total_count']

In [ ]:
x_train, x_test, y_train, y_test_count = train_test_split(x, y, test_size = 0.4, random_state = 0)

In [ ]:
y_pred_count =y_predicted_d_registered + y_predicted_d_casual

In [ ]:
mse = MAE(y_test_count,y_pred_count)
mape = MAPE(y_test_count,y_pred_count)
print("Mean Squared Error:",mse)
print("Mean Absolute Percentage Error:",mape)

In [ ]:
from scipy.stats import pearsonr
corr, _ = pearsonr(y_pred_count,y_test_count)
print(corr)
print(corr**2)

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_pred_count,y_test_count)

In [ ]:
Result by directly using Decision Tree( It automatically selects important features)

For Registered

For Casual

To know more about the analysis, check out "Bike Rental Service.docx" under folder daybikerentalcount